
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 3 -- LP Duality and the Primal-Dual Algorithm** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Jimmy** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 26/2/2020** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Qufei Wang, 19900212-6952, qufei@student.chalmers.se <br />
$\qquad$ $\qquad$$\qquad$$\qquad$$\qquad$ Markus Ingvarsson, 19940712-0154, markusi@student.chalmers.se


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&2x_1 + \ \ {}x_2 - 2x_3 - \ \ x_4 - 3x_5 &&\geq -1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&= 5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + \ \ x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

* (4 points) Write the LP dual of this problem.
* (3 points) Use CVX to compute the primal and dual optimum solutions and compare their values.
* (3 points) Check the complementary slackness conditions.

1.1 The LP dual of this problem is:
\begin{equation}
min\; 6y_1 - y_2 + 5y_3 + 4y_4 \\
\textrm{s.t} \quad 2y_1 + 2y_2 +5y_3 + 2y_4 \geq 4 \\
      2y_1 + y_2 - 2y_3 - 2y_4 \geq -2 \\
      -4y_1 - 2y_2 + 4y_3 + 5y_4 \geq 5 \\
      4y_1 - y_2 + 4y_3 + 3y_4 \geq 6 \\
      8y_1 -3y_2 + 2y_3 + y_4 \geq 7 \\
      y_1 \geq 0 \\
      y_2 \leq 0 \\
      y_4 \geq 0 \\
\end{equation}

In [1]:
# same approach from assignment 1
import numpy as np
import cvxopt.modeling as cv
from IPython.core.debugger import set_trace
from cvxopt import matrix

def calcPrimalDual(A, b, c, matrixCondition, varCondition):
    rows = A.size[0]
    columns = A.size[1]
    primalVars = cv.variable(columns, "primalVars")
    dualVars = cv.variable(rows, "dualVars")
    
    print("Calculate the primal problem:")
    # we always assume that the primal linear program aims to maximum the value. Because method 'cvxopt.modeling.op'
    # is designed to minimize an objective value, so here we need to negate parameter 'c'
    c_primal = -1 * c
    obj_primal = cv.dot(c_primal, primalVars)
    constraints_primal = []
    constraints_primal += matrixConstraints(A, primalVars, b, matrixCondition)
    constraints_primal.append(variableConstraints(primalVars, varCondition))
    lp_primal = cv.op(obj_primal, constraints_primal)
    lp_primal.solve()
    print("Status:")
    print(lp_primal.status)
    print("Variables:")
    print(primalVars)
    print("Optimal value:")
    print(lp_primal.objective.value() * -1)

    print("Calculate the dual problem:")
    # we always assume that the primal linear program aims to maximum the value. Because method 'cvxopt.modeling.op'
    # is designed to minimize an objective value, so here we need to negate parameter 'c'
    obj_dual = cv.dot(b, dualVars)
    constraints_dual = []
    constraints_dual += matrixConstraints(A.trans(), dualVars, c, varCondition)
    constraints_dual.append(variableConstraints(dualVars, -1 * matrixCondition))
    lp_dual = cv.op(obj_dual, constraints_dual)
    lp_dual.solve()
    print("Status:")
    print(lp_dual.status)
    print("Variables:")
    print(dualVars)
    print("Optimal value:")
    print(lp_dual.objective.value())
    
    return primalVars, dualVars
    
    


def matrixConstraints(A, variables, b, matrixCondition):
    result = []
    for i in range(len(matrixCondition)):
        rowI = A[i, :]
        if matrixCondition[i] == -1:
            result.append(rowI * variables <= b[i])
        elif matrixCondition[i] == 1:
            result.append(rowI * variables >= b[i])
        else:
            result.append(rowI * variables == b[i])
    return result
    

    
def variableConstraints(variables, varCondition):
    size = len(variables)
    m = matrix(0, (size, size), 'd')
    for i in range(len(varCondition)):
        if varCondition[i] == 1:
            m[i, i] = 1
        elif varCondition[i] == -1:
            m[i, i] = -1
    return (m * variables >= 0)


In [2]:
# 1.2
A = matrix([[2, 2, 5, 2],
    [2, 1, -2, -2],
    [-4, -2, 4, 5],
    [4, -1, 4, 3],
    [8, -3, 2, 1]], (4, 5), 'd')
b = matrix([6, -1, 5, 4], (4, 1), 'd')
c = matrix([4, -2, 5, 6, 7], (5, 1), 'd')
matrixCondition = np.array([-1, 1, 0, -1], dtype = float)
varCondition = np.array([1, 1, 1, 1, 1], dtype = float)

primalVars, dualVars = calcPrimalDual(A, b, c, matrixCondition, varCondition)

Calculate the primal problem:
     pcost       dcost       gap    pres   dres   k/t
 0: -8.0206e+00 -2.1734e+01  2e+01  5e-01  1e+00  1e+00
 1: -8.5557e+00 -1.0861e+01  2e+00  1e-01  3e-01  5e-01
 2: -9.1823e+00 -9.4031e+00  2e-01  8e-03  2e-02  6e-03
 3: -9.2199e+00 -9.2224e+00  2e-03  9e-05  2e-04  6e-05
 4: -9.2203e+00 -9.2204e+00  2e-05  9e-07  2e-06  6e-07
 5: -9.2203e+00 -9.2203e+00  2e-07  9e-09  2e-08  6e-09
Optimal solution found.
Status:
optimal
Variables:
primalVars: variable of length 5
value: 
[ 6.27e-01]
[ 2.81e+00]
[ 1.54e+00]
[-5.24e-09]
[ 6.61e-01]

Optimal value:
[ 9.22e+00]

Calculate the dual problem:
     pcost       dcost       gap    pres   dres   k/t
 0:  8.0206e+00  3.4476e+01  2e+01  3e-01  3e+00  1e+00
 1:  8.2404e+00  1.3620e+01  2e+00  6e-02  6e-01  5e-01
 2:  9.1930e+00  9.5881e+00  2e-01  5e-03  5e-02  6e-03
 3:  9.2201e+00  9.2244e+00  2e-03  5e-05  5e-04  6e-05
 4:  9.2203e+00  9.2204e+00  2e-05  5e-07  5e-06  6e-07
 5:  9.2203e+00  9.2203e+00  2e-07  5

In [4]:
# 1.3
print("We check the complementary slackness conditions by observing the values of the variables of the dual problem")
print("")
val_primal = A * primalVars
val_dual = A.trans() * dualVars

for i in range(len(dualVars)):
    if abs(dualVars.value[i] - 0) > 0.0000001:
        print("the " + str(i + 1) + "th variable is not zero, so the constraints of the primal problem with index " + str(i + 1) + " should be equality")
        print("the right hand side of constraint " + str(i + 1) + " is:" + str(b[i]))
        print("the actual value of constraint " + str(i + 1) + " is:" + str(val_primal[i].value()))

We check the complementary slackness conditions by observing the values of the variables of the dual problem

the 1th variable is not zero, so the constraints of the primal problem with index 1 should be equality
the right hand side of constraint 1 is:6.0
the actual value of constraint 1 is:[ 6.00e+00]

the 2th variable is not zero, so the constraints of the primal problem with index 2 should be equality
the right hand side of constraint 2 is:-1.0
the actual value of constraint 2 is:[-1.00e+00]

the 3th variable is not zero, so the constraints of the primal problem with index 3 should be equality
the right hand side of constraint 3 is:5.0
the actual value of constraint 3 is:[ 5.00e+00]

the 4th variable is not zero, so the constraints of the primal problem with index 4 should be equality
the right hand side of constraint 4 is:4.0
the actual value of constraint 4 is:[ 4.00e+00]



# Problem 2.

Consdier the LP problem:
\begin{alignat*}{2}
\max \ &6x_1 - 5x_3\\
\\
\textrm{s.t} \quad
&6x_1 - 3x_2 + x_3 &&= 2\\
&3x_1 + 4x_2 + x_3 &&\leq 5\\
&\ \ x_1 - 7x_2 &&\leq 5\\
&\hspace{2.45cm} x_1 &&\geq 0\\
&\hspace{2.45cm} x_2 &&\leq 0\\
&\hspace{2.45cm} x_3 &&\text{ unrestricted}
\end{alignat*}

* (3 points) Write the LP dual of this problem.
* (4 points) Consider the feasible solution $\vec x = (0,0,2)$ to the primal. Check if this solution is optimal by using the complementary slackness conditions to write down the corresponding dual solution.
* (3 points) Use complementary slackness to check if the primal feasible solution $\vec x = (1,0,-4)$ is optimal.

2.1 The LP dual of this problem is:
\begin{equation}
min\; 2y_1 + 5y_2 + 5y_3\\
\textrm{s.t} \quad 6y_1 + 3y_2 +y_3 \geq 6 \\
      -3y_1 + 4y_2 - 7y_3 \leq 0 \\
      y_1 + y_2 = -5 \\
      y_2 \geq 0 \\
      y_3 \geq 0 \\
\end{equation}

2.2 Suppose $\vec x = (0,0,2)$ is optimal, then by the theorm of complementary slackness, we check the variables of its dual problem by seeing whether the constraints of the primal problem bind.
* The first constraint $6x_1 - 3x_2 + x_3 = 2$ binds.
* The second constraint $3x_1 + 4x_2 + x_3 \leq 5$ doesn't bind, as $2 \leq 5$.So we infer that $y_2 == 0$
* The thrid constraint $x_1 - 7x_2 \leq 5$ doesn't bind, as $0 \leq 5$.So we infer that $y_3 == 0$
* With $y_2 == 0$ and $y_3 == 0$, to satisfy $y_1 + y_2 = -5$, $y_1$ must be -5. But this violates the second constraint of the dual problem, in which $-3y_1 + 4y_2 - 7y_3 = 15 > 0$

By using complementary slackness theorem, we know that $\vec x = (0,0,2)$ is not an optimal solution of the primal problem.

2.3 Suppose $\vec x = (1,0,-4)$ is optimal, then by the theorm of complementary slackness, we check the variables of its dual problem by seeing whether the constraints of the primal problem bind.
* The first constraint $6x_1 - 3x_2 + x_3 = 2$ binds.
* The second constraint $3x_1 + 4x_2 + x_3 \leq 5$ doesn't bind, as $-1 < 5$.So we infer that $y_2 == 0$
* The thrid constraint $x_1 - 7x_2 \leq 5$ doesn't bind, as $1 < 5$.So we infer that $y_3 == 0$
* With $y_2 == 0$ and $y_3 == 0$, to satisfy $y_1 + y_2 = -5$, $y_1$ must be -5. But this violates the second constraint of the dual problem, $-3y_1 + 4y_2 - 7y_3 = 15 > 0$

By using complementary slackness theorem, we know that $\vec x = (1,0,-4)$ is not an optimal solution of the primal problem.

# Problem 3.

Consider the primal-dual algorithm for vertex cover discussed in class.
* (4 points) Run the algorithm by hand on the graph in the figure below (from your previous homework). Show the values of the primal and dual variables at each iteration.
* (6 points) Implement the primal-dual algorithm as a python script to compute (approximate) vertex covers and run it for the random graph $G(200,0.1)$ from the previous homework.

<img src="https://tinyurl.com/tsnuz2c" alt="Drawing" style="width: 180px;"/>

3.1 The primal problem can be shown as follows:
\begin{alignat*}{2}
\min \ &4x_a + 3x_b + x_c + 2x_d + 2x_e + 4x_f\\
\\
\textrm{s.t} \quad
&x_a + x_b &&\geq 1\\
&x_a + x_d &&\geq 1\\
&x_a + x_e &&\geq 1\\
&x_b + x_c &&\geq 1\\
&x_d + x_e &&\geq 1\\
&x_e + x_f &&\geq 1\\
&x_b + x_e &&\geq 1\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}
The dual problem can be shown as follows:
\begin{alignat*}{2}
\max \ &y_{ab} + y_{ad} + y_{ae} + y_{bc} + y_{be} + y_{de} + y_{ef}\\
\\
\textrm{s.t} \quad
&y_{ab} + y_{ad} + y_{ae} &&\leq 4\\
&y_{ab} + y_{bc} + y_{be} &&\leq 3\\
&y_{bc} &&\leq 1\\
&y_{ad} + y_{de} &&\leq 2\\
&y_{ae} + y_{de} + y_{be} + y_{ef} &&\leq 2\\
&y_{ef} &&\leq 4\\
&\hspace{5.3cm} \vec y &&\geq \vec 0
\end{alignat*}
The iteration process can be shown as follows:
1. The values of the dual problems are : $y_{ab} = y_{ad} = y_{ae} = y_{bc} = y_{de} = y_{ef} = y_{be} = 0.5$ and the values of the dual problems are $x_e = 1, x_a = x_b = x_c = x_d = x_f = 0$
2. The values of the dual problems are : $y_{ae} = y_{de} = y_{be} = y_{ef} = 0.5$, $y_{bc} = y_{ab} = y_{ad} = 1$ and the values of the dual problems are $x_e = x_c = 1, x_a = x_b = x_d = x_f = 0$
3. The values of the dual problems are : $y_{ae} = y_{de} = y_{be} = y_{ef} = 0.5$, $y_{bc} = 1$, $y_{ab} = y_{ad} = 1.5$ and the values of the dual problems are $x_e = x_c = x_b = x_d = 1, x_a = x_f = 0$

Then all edges ($y_{ab}, \dots, y_{ef}$) are inactive and the algorithm terminates. The objective value we get by our algorithm is 8 while the optimal value is 7.

In [12]:
#3.2
import numpy as np
from cvxopt import matrix

SIZE = 100

def toMatrix(text):
    m = []
    lines = text.splitlines()
    for i in range(len(lines)):
        line = lines[i]
        a = np.fromstring(line, dtype=int, sep=' ')
        for j in range(i, len(a)):
            if a[j] == 1:
                v = buildVector(i, j)
                m.append(v)
    return matrix(np.array(m))


def buildVector(i, j):
    v = np.zeros(SIZE, dtype=float)
    v[i] = v[j] = 1.
    return v

def updateVertex(dictionary, idx, v, edges):
    vertex = dictionary[idx]
    weight = vertex[0]
    numberOfActiveEdges = len(vertex[1])
    weightUpdated = weight - numberOfActiveEdges * v
    edgesUpdated = (vertex[1] - edges)
    return (weightUpdated, edgesUpdated)

            
f = open("graph.txt","r")
text = f.read()
m = toMatrix(text)
rows = m.size[0]
columns = m.size[1]

inactiveEdges = set()
choosenVertices = set()

dictionary = {}

for j in range(columns):
    edges = set()
    for i in range(rows):
        if m[i, j] == 1:
            edges.add(i)
    dictionary[j] = (1, edges)


while len(inactiveEdges) < rows:
    vertexIdx = -1
    tmp = 1;
    edges = None
    for i in range(columns):
        if i not in choosenVertices:
            vertex = dictionary[i]
            weight = vertex[0]
            numberOfActiveEdges = len(vertex[1])
            if numberOfActiveEdges != 0:
                div = weight / numberOfActiveEdges
                if div < tmp:
                    tmp = div
                    vertexIdx = i
                    edges = vertex[1]
    choosenVertices.add(vertexIdx)
    inactiveEdges |= edges
    vtx = updateVertex(dictionary, vertexIdx, tmp, edges)
    dictionary[vertexIdx] = vtx
    for i in range(columns):
        if i not in choosenVertices:
            v = updateVertex(dictionary, i, tmp, edges)
            dictionary[i] = v

print("Choosen vertices:")
print(choosenVertices)
print("")

print("A closer look over the result of all vertices(the left part of the tuple is the left-over weight of a vertex,")
print("the right part of the tuple is the set of the active edges, which are all empty since all edges turn to inactive in the end):")
print("")
for i in dictionary:
    print(dictionary[i])


Choosen vertices:
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 15, 16, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 65, 67, 68, 70, 73, 74, 75, 76, 78, 79, 80, 81, 82, 85, 86, 88, 90, 91, 92, 94, 95, 96, 97, 98, 99}

A closer look over the result of all vertices(the left part of the tuple is the left-over weight of a vertex,
the right part of the tuple is the set of the active edges, which are all empty since all edges turn to inactive in the end):

(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.43315962403773145, set())
(0.060944343949441114, set())
(0.0, set())
(0.0, set())
(0.04419637515419248, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.6956341161074102, set())
(0.0, set())
(0.0, set())
(0.191393580908881, set())
(0.0, set())
(0.0, set())
(0.0, set())
(0.1052751959945224, set())
(0.0, set())
(0.0,